In [42]:
import numpy as np
import pandas as pd
from numpy import nan
import math

In [2]:
df = pd.read_csv('data/Restaurant_Scores_-_LIVES_Standard.csv')

In [29]:
#df.head()

## Basic information about the data

In [4]:
print('shape of data: {}'.format(df.shape))

shape of data: (51297, 17)


In [30]:
#df.info()

In [52]:
colnames = df.columns
for colname in colnames:
    count = df[colname].count()
    print('{}: {}'.format(colname, count))

business_id: 51297
business_name: 51297
business_address: 51297
business_city: 51297
business_state: 51297
business_postal_code: 50010
business_latitude: 30296
business_longitude: 30296
business_location: 30296
business_phone_number: 15917
inspection_id: 51297
inspection_date: 51297
inspection_score: 38135
inspection_type: 51297
violation_id: 38728
violation_description: 38728
risk_category: 38728


### Summary of the data
1. The total number of rows is 51297. 
2. Violations id's, violation descriptions, and risk categories are 38728. 
3. 1287 postal codes are missing: 50010
4. Lots of latitudes, longitues, and location are missing: 30296
5. Lots of inspection scores are missing: 38135
6. Violation id's are missing for reinspection/followup, new construction, new ownership,

### Look for inspection types for missing violation id's

In [7]:
df2 = df[['violation_id', 'inspection_type']]

In [8]:
mask = df2['violation_id'].isnull()

In [9]:
df3 = df2[mask]
inspect_types = set()
for atype in df3['inspection_type']:
    inspect_types.add(atype)

In [32]:
#print("Inpection types for missing violation id's: \n{}".format(inspect_types))

## Remove the missing violation id rows. 

In [11]:
# They are not the ones issued with violations.

In [12]:
mask_viol = df['violation_id'].isnull()

In [13]:
mask_viol.shape

(51297,)

In [14]:
df_viol = df[~mask_viol]

In [15]:
df_viol.shape

(38728, 17)

In [33]:
#df_viol.head()

## Clean up the violation id's. 

In [17]:
# 1757_20170928_103131: The first two series of numbers are the inspection id.

In [18]:
df_clean = df_viol[:]

In [19]:
a = df_clean['violation_id']

In [36]:
df_clean.head(20)

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,short_violation_id
0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",NaN,1757_20170928,9/28/2017 0:00,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk,103131
1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",1.415583e+10,4864_20161206,12/6/2016 0:00,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk,103157
2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,NaN,NaN,NaN,79782_20160503,5/3/2016 0:00,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk,103120
3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,NaN,NaN,NaN,73840_20171207,12/7/2017 0:00,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk,103105
4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,NaN,NaN,NaN,76437_20160329,3/29/2016 0:00,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk,103113
5,82123,Cafe New Honolulu,888 Stockton St,San Francisco,CA,94108,NaN,NaN,NaN,NaN,82123_20171207,12/7/2017 0:00,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk,103131
6,1146,Aux Delices Vietnam,2327 Polk St,San Francisco,CA,94109,37.798274,-122.422264,"(37.798274, -122.422264)",NaN,1146_20180223,2/23/2018 0:00,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk,103103
7,3838,CAFE PICARO,3120 16th St,San Francisco,CA,94103,37.764908,-122.422442,"(37.764908, -122.422442)",1.415543e+10,3838_20180222,2/22/2018 0:00,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk,103142
8,38929,Jelly Donut,3198 24th St,San Francisco,CA,94110,37.752439,-122.416250,"(37.752439, -122.41625)",NaN,38929_20160114,1/14/2016 0:00,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk,103139
9,81161,Limon Peruvian Rotisserie,1001 South Van Ness Ave,San Francisco,CA,94110,NaN,NaN,NaN,1.415555e+10,81161_20160325,3/25/2016 0:00,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk,103149


In [21]:
a[0].split('_')[2]

'103131'

In [22]:
L_vid = []
for id in a:
    vid = id.split('_')[2]
    L_vid.append(vid)
ps_vid = pd.Series(L_vid)

In [53]:
# number of violation ids without inspection id in front. this should be 38728
len(ps_vid)

38728

In [55]:
# is there any NaN values?
ps_vid.isnull().any()

False

In [25]:
df_clean['short_violation_id'] = ps_vid

/home/soong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
# is there any NaN in df_clean['short_violation_id'] column
df_clean['short_violation_id'].isnull().any()

True

In [60]:
# let's find out why there are NaN values by comparing violation ids and short violation ids
mask_clean = df_clean['short_violation_id'].isnull()
len(mask_clean)

38728

In [63]:
df_clean_diag = df_clean[mask_clean]
df_clean_diag.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,short_violation_id
38729,39667,Peet's Coffee & Tea,595 Market St Suite #143,San Francisco,CA,94105,37.789437,-122.401111,"(37.789437, -122.401111)",NaN,39667_20170912,9/12/2017 0:00,82.0,Routine - Unscheduled,39667_20170912_103142,Unclean nonfood contact surfaces,Low Risk,NaN
38730,91289,Lapats Thai Noodles Bar,601 Larkin St,San Francisco,CA,94109,NaN,NaN,NaN,NaN,91289_20170905,9/5/2017 0:00,89.0,Routine - Unscheduled,91289_20170905_103144,Unapproved or unmaintained equipment or utensils,Low Risk,NaN
38731,89059,Nande-Ya,1737 Post St 375,San Francisco,CA,94115,NaN,NaN,NaN,1.415369e+10,89059_20170227,2/27/2017 0:00,72.0,Routine - Unscheduled,89059_20170227_103133,Foods not protected from contamination,Moderate Risk,NaN
38732,77645,Banana Home,321 Kearny St,San Francisco,CA,94108,NaN,NaN,NaN,NaN,77645_20170320,3/20/2017 0:00,85.0,Routine - Unscheduled,77645_20170320_103133,Foods not protected from contamination,Moderate Risk,NaN
38733,66536,Cypress Grill,99 Harding Park Rd,San Francisco,CA,94132,37.723698,-122.495289,"(37.723698, -122.495289)",1.415566e+10,66536_20170303,3/3/2017 0:00,90.0,Routine - Unscheduled,66536_20170303_103149,Wiping cloths not clean or properly stored or ...,Low Risk,NaN


In [68]:
a53 = df_clean_diag['violation_id']

In [74]:
b53 = a53.iloc[0]

In [75]:
b53.split('_')[2]

'103142'

In [26]:
df_clean.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,short_violation_id
0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",NaN,1757_20170928,9/28/2017 0:00,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk,103131
1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",1.415583e+10,4864_20161206,12/6/2016 0:00,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk,103157
2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,NaN,NaN,NaN,79782_20160503,5/3/2016 0:00,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk,103120
3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,NaN,NaN,NaN,73840_20171207,12/7/2017 0:00,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk,103105
4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,NaN,NaN,NaN,76437_20160329,3/29/2016 0:00,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk,103113


## Creating 7 time periods
y label time period (past 6 months) and features of violations for 6 different time periods:  
0) 11/2017 - 3/2017: y label (0 or 1)  
i) 8/2017 - 10/2017: first period  
ii) 5/2017 - 7/2017: second period  
iii) 2/2017 - 4/2017: third period  
iv) 11/2016 - 1/2017: fourth period  
v) 5/2016 - 10/2016: fifth period  
vi) 3/2015 - 4/2016: sixth period  

In [28]:
df_periods = df_clean[:]

In [44]:
df_periods.to_csv('data/df_periods.csv')

In [48]:
df_periods['short_violation_id'].isnull().sum()

9336